In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
frame = pd.read_csv('../data/PublicTransitExpenses.csv')

C:\Users\Dawid\AppData\Local\Temp\ipykernel_3956\3064211645.py:1: DtypeWarning: Columns (21,22,23,24,25,26,27,28,29,30,31,32,33,37) have mixed types. Specify dtype option on import or set low_memory=False.
  frame = pd.read_csv('../data/PublicTransitExpenses.csv')


In [3]:
frame = pd.read_csv('../data/PublicTransitExpenses.csv', low_memory=False)

low_memory=False argument will turn off pandas optimization while using memory during import. 
If columns have mixed data types, some data can be damaged

In [4]:
frame.head()

,5 digit NTD ID,4 digit NTD ID,Agency,Reporter Type,Subrecipient Type,Organization Type,2015 Total Mode Vehicles (VOMS),Mode,Type of Service,Primary UZA Code,...,Casualty and Liability Costs,Taxes,PT Funds In Report,PT Funds Reported Separately,Miscellaneous Expenses,Reduced Reporter Total OE,Total Operating Expenses,Total Operating Expenses (No Funds Reported Separately),ADA Related Expenses,Location 1
0,30098,3098,Washington County Commissioners,Reduced Reporter,NaN,NaN,NaN,DR,PT,NaN,...,$0.00,$0.00,$0.00,$0.00,$0.00,$122524.00,$122524.00,$122524.00,$0.00,NaN
1,30098,3098,Washington County Commissioners,Reduced Reporter,NaN,NaN,NaN,MB,PT,NaN,...,$0.00,$0.00,$0.00,$0.00,$0.00,$272715.00,$272715.00,$272715.00,$0.00,NaN
2,60107,6107,"Texoma Area Paratransit System, Inc",Full Reporter,NaN,NaN,NaN,CB,PT,NaN,...,NaN,NaN,$3398.00,NaN,NaN,NaN,$7295.00,$7295.00,NaN,NaN
3,9,0T09,Kalispel Tribe of Indians,Reduced Reporter,NaN,Tribe,4.0,DR,DO,0.0,...,NaN,NaN,NaN,NaN,NaN,$37416.00,$37416.00,$37416.00,NaN,"Usk, WA\n"
4,9,0T09,Kalispel Tribe of Indians,Reduced Reporter,NaN,Tribe,4.0,MB,DO,0.0,...,NaN,NaN,NaN,NaN,NaN,$345789.00,$345789.00,$345789.00,NaN,"Usk, WA\n"


In [5]:
frame.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17844 entries, 0 to 17843
Data columns (total 39 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   5 digit NTD ID                                           17844 non-null  object 
 1   4 digit NTD ID                                           17719 non-null  object 
 2   Agency                                                   17844 non-null  object 
 3   Reporter Type                                            17844 non-null  object 
 4   Subrecipient Type                                        3072 non-null   object 
 5   Organization Type                                        17759 non-null  object 
 6   2015 Total Mode Vehicles (VOMS)                          17522 non-null  float64
 7   Mode                                                     17844 non-null  object 
 8   Type of Service           

Most data types are not optimal - objects

Import only relevant data:

In [6]:
frame = pd.read_csv('../data/PublicTransitExpenses.csv', 
                    low_memory=False, 
                    usecols=['Agency', 'Reporter Type', 'Organization Type', 
                             'Rail (Y/N)', 'Fixed Route (Y/N)', 'Service Costs',
                             'Tires and Tubes', 'Total Operating Expenses', 'Service Area Population'])

In [7]:
frame.head()

,Agency,Reporter Type,Organization Type,Service Area Population,Rail (Y/N),Fixed Route (Y/N),Service Costs,Tires and Tubes,Total Operating Expenses
0,Washington County Commissioners,Reduced Reporter,NaN,NaN,NaN,NaN,$0.00,$0.00,$122524.00
1,Washington County Commissioners,Reduced Reporter,NaN,NaN,NaN,NaN,$0.00,$0.00,$272715.00
2,"Texoma Area Paratransit System, Inc",Full Reporter,NaN,NaN,NaN,NaN,NaN,NaN,$7295.00
3,Kalispel Tribe of Indians,Reduced Reporter,Tribe,NaN,N,N,NaN,NaN,$37416.00
4,Kalispel Tribe of Indians,Reduced Reporter,Tribe,NaN,N,Y,NaN,NaN,$345789.00


In [8]:
frame.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17844 entries, 0 to 17843
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Agency                    17844 non-null  object 
 1   Reporter Type             17844 non-null  object 
 2   Organization Type         17759 non-null  object 
 3   Service Area Population   14418 non-null  float64
 4   Rail (Y/N)                17318 non-null  object 
 5   Fixed Route (Y/N)         17318 non-null  object 
 6   Service Costs             10262 non-null  object 
 7   Tires and Tubes           5615 non-null   object 
 8   Total Operating Expenses  17844 non-null  object 
dtypes: float64(1), object(8)
memory usage: 9.5 MB


One can rename columns if their names are long and hard to work with

In [38]:
newColumnNames = {
    'Agency'                    : 'Agency',
    'Reporter Type'             : 'ReporterType',
    'Organization Type'         : 'OrgType',
    'Service Area Population'   : 'Population',
    'Rail (Y/N)'                : 'IsRail',
    'Fixed Route (Y/N)'         : 'IsFixedRoute',
    'Service Costs'             : 'ServiceCosts',
    'Tires and Tubes'           : 'TiresTubesCost',
    'Total Operating Expenses'  : 'TotalExpenses'
}

In [39]:
frame.rename(columns=newColumnNames, inplace=True)

In [15]:
frame.head()

,Agency,ReporterType,OrgType,Population,IsRail,Fixed Route (Y/N),ServiceCosts,TiresTubesCost,TotalExpenses
0,Washington County Commissioners,Reduced Reporter,NaN,NaN,NaN,NaN,$0.00,$0.00,$122524.00
1,Washington County Commissioners,Reduced Reporter,NaN,NaN,NaN,NaN,$0.00,$0.00,$272715.00
2,"Texoma Area Paratransit System, Inc",Full Reporter,NaN,NaN,NaN,NaN,NaN,NaN,$7295.00
3,Kalispel Tribe of Indians,Reduced Reporter,Tribe,NaN,N,N,NaN,NaN,$37416.00
4,Kalispel Tribe of Indians,Reduced Reporter,Tribe,NaN,N,Y,NaN,NaN,$345789.00


One can use nunique, count and / or value_counts methods to find wheather he shall change data type to category or not

In [16]:
frame.ReporterType.nunique()

4

In [18]:
frame.ReporterType.count()

17844

In [19]:
frame.ReporterType.value_counts()

ReporterType
Full Reporter       13345
Rural Reporter       3116
Reduced Reporter     1313
Separate Service       70
Name: count, dtype: int64

In [20]:
frame.ReporterType = frame.ReporterType.astype('category')

In [21]:
frame.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17844 entries, 0 to 17843
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Agency             17844 non-null  object  
 1   ReporterType       17844 non-null  category
 2   OrgType            17759 non-null  object  
 3   Population         14418 non-null  float64 
 4   IsRail             17318 non-null  object  
 5   Fixed Route (Y/N)  17318 non-null  object  
 6   ServiceCosts       10262 non-null  object  
 7   TiresTubesCost     5615 non-null   object  
 8   TotalExpenses      17844 non-null  object  
dtypes: category(1), float64(1), object(7)
memory usage: 8.4 MB


In [22]:
frame.Agency = frame.Agency.astype('category')
frame.OrgType = frame.OrgType.astype('category')

In [23]:
frame.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17844 entries, 0 to 17843
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Agency             17844 non-null  category
 1   ReporterType       17844 non-null  category
 2   OrgType            17759 non-null  category
 3   Population         14418 non-null  float64 
 4   IsRail             17318 non-null  object  
 5   Fixed Route (Y/N)  17318 non-null  object  
 6   ServiceCosts       10262 non-null  object  
 7   TiresTubesCost     5615 non-null   object  
 8   TotalExpenses      17844 non-null  object  
dtypes: category(3), float64(1), object(5)
memory usage: 5.2 MB


In [32]:
frame.Population = frame.Population.fillna(0)
frame.Population = frame['Population'].astype('int')

In [33]:
frame.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17844 entries, 0 to 17843
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   Agency             17844 non-null  category
 1   ReporterType       17844 non-null  category
 2   OrgType            17759 non-null  category
 3   Population         17844 non-null  int32   
 4   IsRail             17318 non-null  object  
 5   Fixed Route (Y/N)  17318 non-null  object  
 6   ServiceCosts       10262 non-null  object  
 7   TiresTubesCost     5615 non-null   object  
 8   TotalExpenses      17844 non-null  object  
dtypes: category(3), int32(1), object(5)
memory usage: 5.1 MB


In [40]:
frame.head()

,Agency,ReporterType,OrgType,Population,IsRail,IsFixedRoute,ServiceCosts,TiresTubesCost,TotalExpenses
0,Washington County Commissioners,Reduced Reporter,NaN,0,NaN,NaN,$0.00,$0.00,$122524.00
1,Washington County Commissioners,Reduced Reporter,NaN,0,NaN,NaN,$0.00,$0.00,$272715.00
2,"Texoma Area Paratransit System, Inc",Full Reporter,NaN,0,NaN,NaN,NaN,NaN,$7295.00
3,Kalispel Tribe of Indians,Reduced Reporter,Tribe,0,N,N,NaN,NaN,$37416.00
4,Kalispel Tribe of Indians,Reduced Reporter,Tribe,0,N,Y,NaN,NaN,$345789.00


In [43]:
frame.IsRail = frame.IsRail.replace(['Y', 'N'], [True, False])
frame.IsFixedRoute = frame.IsFixedRoute.replace(['Y', 'N'], [True, False])

In [44]:
frame.head()

,Agency,ReporterType,OrgType,Population,IsRail,IsFixedRoute,ServiceCosts,TiresTubesCost,TotalExpenses
0,Washington County Commissioners,Reduced Reporter,NaN,0,NaN,NaN,$0.00,$0.00,$122524.00
1,Washington County Commissioners,Reduced Reporter,NaN,0,NaN,NaN,$0.00,$0.00,$272715.00
2,"Texoma Area Paratransit System, Inc",Full Reporter,NaN,0,NaN,NaN,NaN,NaN,$7295.00
3,Kalispel Tribe of Indians,Reduced Reporter,Tribe,0,False,False,NaN,NaN,$37416.00
4,Kalispel Tribe of Indians,Reduced Reporter,Tribe,0,False,True,NaN,NaN,$345789.00


In [47]:
frame.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17844 entries, 0 to 17843
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Agency          17844 non-null  category
 1   ReporterType    17844 non-null  category
 2   OrgType         17759 non-null  category
 3   Population      17844 non-null  int32   
 4   IsRail          17844 non-null  bool    
 5   IsFixedRoute    17844 non-null  bool    
 6   ServiceCosts    10262 non-null  object  
 7   TiresTubesCost  5615 non-null   object  
 8   TotalExpenses   17844 non-null  object  
dtypes: bool(2), category(3), int32(1), object(3)
memory usage: 3.2 MB


In [48]:
frame.IsRail = frame.IsRail.fillna(False).astype('bool')
frame.IsFixedRoute = frame.IsFixedRoute.fillna(False).astype('bool')

In is not always wise to switch NaN to T/F

In [49]:
frame.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17844 entries, 0 to 17843
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Agency          17844 non-null  category
 1   ReporterType    17844 non-null  category
 2   OrgType         17759 non-null  category
 3   Population      17844 non-null  int32   
 4   IsRail          17844 non-null  bool    
 5   IsFixedRoute    17844 non-null  bool    
 6   ServiceCosts    10262 non-null  object  
 7   TiresTubesCost  5615 non-null   object  
 8   TotalExpenses   17844 non-null  object  
dtypes: bool(2), category(3), int32(1), object(3)
memory usage: 3.2 MB


In [50]:
frame.head()

,Agency,ReporterType,OrgType,Population,IsRail,IsFixedRoute,ServiceCosts,TiresTubesCost,TotalExpenses
0,Washington County Commissioners,Reduced Reporter,NaN,0,False,False,$0.00,$0.00,$122524.00
1,Washington County Commissioners,Reduced Reporter,NaN,0,False,False,$0.00,$0.00,$272715.00
2,"Texoma Area Paratransit System, Inc",Full Reporter,NaN,0,False,False,NaN,NaN,$7295.00
3,Kalispel Tribe of Indians,Reduced Reporter,Tribe,0,False,False,NaN,NaN,$37416.00
4,Kalispel Tribe of Indians,Reduced Reporter,Tribe,0,False,True,NaN,NaN,$345789.00


In [57]:
frame.ServiceCosts = frame.ServiceCosts.str.replace('$', '')
frame.TiresTubesCost = frame.TiresTubesCost.str.replace('$', '')
frame.TotalExpenses = frame.TotalExpenses.str.replace('$', '')

In [58]:
frame.head()

,Agency,ReporterType,OrgType,Population,IsRail,IsFixedRoute,ServiceCosts,TiresTubesCost,TotalExpenses
0,Washington County Commissioners,Reduced Reporter,NaN,0,False,False,0.00,0.00,122524.00
1,Washington County Commissioners,Reduced Reporter,NaN,0,False,False,0.00,0.00,272715.00
2,"Texoma Area Paratransit System, Inc",Full Reporter,NaN,0,False,False,NaN,NaN,7295.00
3,Kalispel Tribe of Indians,Reduced Reporter,Tribe,0,False,False,NaN,NaN,37416.00
4,Kalispel Tribe of Indians,Reduced Reporter,Tribe,0,False,True,NaN,NaN,345789.00


In [59]:
frame.ServiceCosts = frame.ServiceCosts.astype('float')
frame.TiresTubesCost = frame.TiresTubesCost.astype('float')
frame.TotalExpenses = frame.TotalExpenses.astype('float')

In [60]:
frame.head()

,Agency,ReporterType,OrgType,Population,IsRail,IsFixedRoute,ServiceCosts,TiresTubesCost,TotalExpenses
0,Washington County Commissioners,Reduced Reporter,NaN,0,False,False,0.0,0.0,122524.0
1,Washington County Commissioners,Reduced Reporter,NaN,0,False,False,0.0,0.0,272715.0
2,"Texoma Area Paratransit System, Inc",Full Reporter,NaN,0,False,False,NaN,NaN,7295.0
3,Kalispel Tribe of Indians,Reduced Reporter,Tribe,0,False,False,NaN,NaN,37416.0
4,Kalispel Tribe of Indians,Reduced Reporter,Tribe,0,False,True,NaN,NaN,345789.0


In [61]:
frame.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17844 entries, 0 to 17843
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Agency          17844 non-null  category
 1   ReporterType    17844 non-null  category
 2   OrgType         17759 non-null  category
 3   Population      17844 non-null  int32   
 4   IsRail          17844 non-null  bool    
 5   IsFixedRoute    17844 non-null  bool    
 6   ServiceCosts    10262 non-null  float64 
 7   TiresTubesCost  5615 non-null   float64 
 8   TotalExpenses   17844 non-null  float64 
dtypes: bool(2), category(3), float64(3), int32(1)
memory usage: 846.4 KB


In [64]:
frame.Agency = frame.Agency.str.title().astype('category')
frame.ReporterType = frame.ReporterType.str.upper().astype('category')
frame.OrgType = frame.OrgType.str.upper().astype('category')

In [65]:
frame.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17844 entries, 0 to 17843
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Agency          17844 non-null  category
 1   ReporterType    17844 non-null  category
 2   OrgType         17759 non-null  category
 3   Population      17844 non-null  int32   
 4   IsRail          17844 non-null  bool    
 5   IsFixedRoute    17844 non-null  bool    
 6   ServiceCosts    10262 non-null  float64 
 7   TiresTubesCost  5615 non-null   float64 
 8   TotalExpenses   17844 non-null  float64 
dtypes: bool(2), category(3), float64(3), int32(1)
memory usage: 846.1 KB


In [66]:
frame.head()

,Agency,ReporterType,OrgType,Population,IsRail,IsFixedRoute,ServiceCosts,TiresTubesCost,TotalExpenses
0,Washington County Commissioners,REDUCED REPORTER,NaN,0,False,False,0.0,0.0,122524.0
1,Washington County Commissioners,REDUCED REPORTER,NaN,0,False,False,0.0,0.0,272715.0
2,"Texoma Area Paratransit System, Inc",FULL REPORTER,NaN,0,False,False,NaN,NaN,7295.0
3,Kalispel Tribe Of Indians,REDUCED REPORTER,TRIBE,0,False,False,NaN,NaN,37416.0
4,Kalispel Tribe Of Indians,REDUCED REPORTER,TRIBE,0,False,True,NaN,NaN,345789.0
